# CSE 5243 - Introduction to Data Mining
## Homework 5: Association Analysis
- Semester: SP24
- Instructor: Professor Bihari
- Section: Wednesday Friday 12:45-2:20
- Student Name: Erik Thompson
- Student Email: thompson.3624@osu.edu
- Student ID: 500367903

Template Version V1.
***

# Introduction

### Objectives

In this lab, we will use the Instacart Market Basket Analysis dataset (see: https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis). However, we will use the data to solve a different (completely nonsense) problem.  The dataset Excel file "**osu_combined_data.xlsx**" provided on Carmen **has been modified** from the original Instacart data files.  The file contains two worksheets:
- **osu_products_with_price**: This is a trimmed down copy of the Instacart **products** data, with an additional, fabricated column added: **fake_unit_price**.  This column contains a random unit price for each of the products.  It is not realistic in any way.
- **osu_order_products_subset**: This is a trimmed down copy of the Instacart **order_products__train** data, with a subset of the total records, and only the **order_id** and **product_id** columns.

The objectives of this assignment are:
- Practice the Association Analysis content we covered this semester.
- Understand “why” the particular topics, techniques, etc., are important from a practical perspective.
- Understand how to choose and use appropriate tools to solve the provided problems.

### Dataset Notes
- The **osu_order_products_subset** dataset captures the data in "long format". Specifically, every row corresponds to the order id and the product id. If the specific order id has multiple products, there will be multiple rows in the data.
- You can process the data however you like, but it is recommended you convert into a one-hot-encoded data structure. This will allow you to easily use the mlxtend package.

## The Business Problem
- Assume the provided dataset contains **all** of the transactions for one month for our store (Trader Buck's).  We wish to find association rules that will improve our revenue as follows:
  - We would discount one of our products by 10% each month, with the hope that this would encourage customers to visit our store to purchase that product 5% more frequently, and also purchase  other associated products (that are not discounted) 5% more frequently.
- Practically speaking, we would like to come up with **two-item** rules (one antecedent and one consequent: (A -> B)) and choose the one that best adds to our revenues  (based on the rule support, confidence, etc.).

### Proper Answers
- **IMPORTANT:** **Show your work** and **explain it**.  This will help us give partial credit in some cases.

### Collaboration
For this assignment, you must work as an individual. You may informally discuss ideas with classmates, but your work must be your own.

### What You Need to Turn In
- Submit this Jupyter Notebook in .IPYNB format.  Do not "zip" the file.

### Notes
- Feel free to use the **mlxtend** package throughout this assignment.
- If a question asks you to "calculate" the number of "all possible rules", etc., explain the calculation by showing the "formula" you used. This will act as "showing your work".
- The terms ("order" and "transaction") and ("product" and "item") are used interchangeably below.
***

## Some Guidance

Lets keep things simple and do / assume the following:
- Generate a set of 2-Itemsets (about 50-100).
- Generate all of the rules that can be made from those 2-Itemsets.  Not surprisingly, you should end up with 2x the number of 2-Itemsets.
- Keep it simple:
  - Don’t consider secondary / chaining impacts: ((Milk -> Eggs) and (Eggs -> Bread) and (Bread -> Cheese)).
  - Don’t consider looping: ((Milk -> Eggs) and (Eggs -> Milk)).  Remember: Correlation is not Causation.
  - Of the remaining rules, process each individual rule separately (and add up the impacts).
    - Count the impact of the Antecedent only once.  That is, don’t count the Milk impact twice if you have two rules:
      - (Milk -> Eggs)
      - (Milk -> Butter)
  - This is a bit over-simplified, but that’s OK.
- If this dataset is too large to run fairly rapidly on your machine, let me know and we can come up with workarounds.  My answer runs in about 3 minutes on my laptop.  Your results may vary.
- Also, you probably should make Discount% (10%) and SalesIncrease% (5%) variables, so you can experiment with changing those values.
- You may have a different way of formulating the problem (and it is an artificial problem anyway). So you may get different results. Just remember to explain what your assumptions / models are, so we can give credit for a variety of answers.
***

***
# Section: 1 - Get Ready
1A) Load the data, and get it ready for association analysis. Do this with convenient python helper methods as appropriate. Feel free to use the tools given in the example we covered. 
- Suggest: Make the data one-hot encoded.
***

In [1]:
!pip install mlxtend

In [2]:
#Note: If the mlxtend library is not installed, uncomment the following line (once) and run it.

import decimal
import numpy as np
import pandas as pd
import mlxtend as mlx
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth

pd.set_option('display.max_columns', 1000) #include to avoid ... in middle of display (and use 'display(...)' when printing in cells)

In [3]:
# load the products data
products_df = pd.read_excel('../Data/InstaCartMBA/osu_combined_data.xlsx', sheet_name='osu_products_with_price')
display(products_df.info())
display(products_df.describe())
products_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   product_id       49688 non-null  int64  
 1   product_name     49687 non-null  object 
 2   fake_unit_price  49688 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB


None

,product_id,fake_unit_price
count,49688.000000,49688.000000
mean,24844.500000,5.487106
std,14343.834425,2.601488
min,1.000000,1.000000
25%,12422.750000,3.240000
50%,24844.500000,5.480000
75%,37266.250000,7.730000
max,49688.000000,9.990000


,product_id,product_name,fake_unit_price
0,1,Chocolate Sandwich Cookies,3.79
1,2,All-Seasons Salt,2.16
2,3,Robust Golden Unsweetened Oolong Tea,7.65
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,6.16
4,5,Green Chile Anytime Sauce,9.41


In [4]:
# load the orders data
orders_df = pd.read_excel('../Data/InstaCartMBA/osu_combined_data.xlsx', sheet_name='osu_order_products_subset')
display(orders_df.info())
display(orders_df.describe())
orders_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 2 columns):
 #   Column      Non-Null Count    Dtype
---  ------      --------------    -----
 0   order_id    1048575 non-null  int64
 1   product_id  1048575 non-null  int64
dtypes: int64(2)
memory usage: 16.0 MB


None

,order_id,product_id
count,1.048575e+06,1.048575e+06
mean,1.289710e+06,2.555930e+04
std,7.483777e+05,1.411837e+04
min,1.000000e+00,1.000000e+00
25%,6.384330e+05,1.339800e+04
50%,1.287873e+06,2.529500e+04
75%,1.935899e+06,3.793700e+04
max,2.593147e+06,4.968800e+04


,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


In [31]:
#Sample Data
df = orders_df.sample(frac=0.02, random_state=4000)

In [35]:
#Sourced from Professor Bihari on Microsoft teams
# One hot encode the Transaction data
# orders_df is a "long form" dataset. We first convert it to "wide form" then one-hot encode it.
from mlxtend.preprocessing import TransactionEncoder
from collections import defaultdict
 
transaction_items = defaultdict(list)
for transaction in orders_df.values.tolist():
    transaction_items[transaction[0]].append(transaction[1])
    
dataset_wide = list(transaction_items.values())
 
te = TransactionEncoder()
te_ary = te.fit(dataset_wide).transform(dataset_wide)
 
ohe_df= pd.DataFrame(te_ary, columns=te.columns_)

***
# Section: 2 - Explore the Data
***

***
## Section: 2.1 - Get the Order and Product Sizes
- Calculate the **number_of_orders** and **number_of_products** (in the orders).
***

In [9]:
number_of_orders = orders_df.shape[0]
print("Number of Orders " + str(number_of_orders))

number_of_products = ohe_df.shape[1]
print("Number of Products " + str(number_of_products))

Number of Orders 1048575
Number of Products 14534


***
## Section: 2.2 - Evaluate the Itemset and Rule Size & Complexity
- Calculate the **maximum number of Itemsets** that could be created from the items (without considering the actual transaction data). Show your work.
- Calculate the **maximum number of Rules** that can be created from the items (without considering the actual transaction data). Show your work.
- What do the calculations suggest as a **potential cause of concern**? Hint: Complexity.
- What might you do to manage these concerns?
***

In [10]:
import decimal
max_itemsets = decimal.Decimal(2 ** number_of_products)
print("Maximum number of Itemsets " + format(max_itemsets, '.6e'))

max_rules = decimal.Decimal((3 ** number_of_products) - (2 ** (number_of_products+1) ) + 1)
print("Maximum number of rules " + format(max_rules, '.6e'))

Maximum number of Itemsets 1.478962e+4375
Maximum number of rules 3.022151e+6934


The calculations suggest a potential cause of concern as very complex number of itemsets. It would be impossible to calculate all number of itemsets. Same goes for number of rules. To manage these concerns only a small amount of itemsets and rules should be calculated and use an algorithm that finds important ones.

***
# Section: 3 - Itemset Generation
***

***
## Section: 3.1 - Revise the Dataset
- If/as appropriate, trim or revise the dataset to make the runtime reasonable.
- Show the results, briefly.
- Explain what you did and why you did it.
***

In [11]:
print(df)

         order_id  product_id
89184      222114       45948
1037736   2565909       33845
923621    2278886       45613
1030330   2547054       42741
810720    1999364       21616
...           ...         ...
412903    1011151       32636
938366    2315508       44632
130940     319464       30391
857356    2116375         165
137690     336157       35951

[10486 rows x 2 columns]


I trimmed the number of orders down to 1/40th of the original size. This is the highest possible on my laptop to make a one hot vector encoding.


In [12]:
print(ohe_df)

       1        10       32       34       35       43       45       49       \
0            0        0        0        0        0        0        0        0   
1            0        0        0        0        0        0        0        0   
2            0        0        0        0        0        0        0        0   
3            0        0        0        0        0        0        0        0   
4            0        0        0        0        0        0        0        0   
...        ...      ...      ...      ...      ...      ...      ...      ...   
10481        0        0        0        0        0        0        0        0   
10482        0        0        0        0        0        0        0        0   
10483        0        0        0        0        0        0        0        0   
10484        0        0        0        0        0        0        0        0   
10485        0        0        0        0        0        0        0        0   

       64       93       10

I made a one hot vector encoding according to the example code given. This allows for multiple product orders to be in one row of the table.

***
## Section: 3.2 - Create Two-Itemsets
- Create a set of about 50 to 100 two-item sets with highest support. Sort them in decreasing order of support.
- Show the results, briefly.
- Explain what you did and why you did it.
***

In [13]:
ohe_df.columns = [str(i) for i in ohe_df.columns]

In [21]:
frequent_itemsets = apriori(ohe_df, min_support=0.0002, use_colnames=True, max_len=2)
print(frequent_itemsets)
two_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x)) == 2]
two_sorted = two_itemsets.sort_values(by='support', ascending=False)

top_two_sorted = two_sorted.head(100)
print(two_sorted)

       support       itemsets
0     0.000205            (1)
1     0.000205           (10)
2     0.000616           (45)
3     0.000205          (116)
4     0.000308          (128)
...        ...            ...
1594  0.000308        (49605)
1595  0.000308        (49610)
1596  0.000205        (49621)
1597  0.001952        (49683)
1598  0.000205  (45007, 4799)

[1599 rows x 2 columns]
       support       itemsets
1598  0.000205  (45007, 4799)


In [36]:
frequent_itemsets = apriori(ohe_df, min_support=0.005, use_colnames=True, max_len=2)
print(frequent_itemsets)
two_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x)) == 2]
two_sorted = two_itemsets.sort_values(by='support', ascending=False)

top_two_sorted = two_sorted.head(100)
print(two_sorted)

      support        itemsets
0    0.011338           (196)
1    0.009460           (260)
2    0.009641           (432)
3    0.006327           (890)
4    0.005825          (1158)
..        ...             ...
360  0.005333  (40706, 47766)
361  0.005534  (47626, 45007)
362  0.005524  (47626, 46979)
363  0.005684  (47209, 47626)
364  0.010123  (47626, 47766)

[365 rows x 2 columns]
      support        itemsets
277  0.023279  (13176, 21137)
292  0.018077  (13176, 47209)
348  0.017002  (24852, 47766)
278  0.016852  (13176, 21903)
302  0.016370  (21137, 24852)
..        ...             ...
359  0.005072  (47626, 30391)
272  0.005052   (26209, 8518)
297  0.005031  (43352, 16797)
295  0.005021  (16797, 21903)
350  0.005011  (26209, 24964)

[107 rows x 2 columns]


***
# Section: 4 - Generate Rules
- For the two-itemsets created above, create the related rules.
***

In [40]:
# Generate association rules
rules = association_rules(top_two_sorted, metric="lift", min_threshold=0.01, support_only=True)

# Sort rules by confidence in descending order
rules_sorted = rules.sort_values(by='confidence', ascending=False)

# Display the association rules
print(rules_sorted)

   antecedents consequents  antecedent support  consequent support   support  \
0      (13176)     (21137)                 NaN                 NaN  0.023279   
1      (21137)     (13176)                 NaN                 NaN  0.023279   
2      (13176)     (47209)                 NaN                 NaN  0.018077   
3      (47209)     (13176)                 NaN                 NaN  0.018077   
4      (24852)     (47766)                 NaN                 NaN  0.017002   
5      (47766)     (24852)                 NaN                 NaN  0.017002   
6      (13176)     (21903)                 NaN                 NaN  0.016852   
7      (21903)     (13176)                 NaN                 NaN  0.016852   
8      (21137)     (24852)                 NaN                 NaN  0.016370   
9      (24852)     (21137)                 NaN                 NaN  0.016370   
10     (47626)     (24852)                 NaN                 NaN  0.016320   
11     (24852)     (47626)              

***
# Section: 5 - Rule Evaluation
- For the rules created above, find the single Item (that would be given the discount) that would cause the greatest increase in monthly store revenue.
  - This is based on the Business Problem stated at the top of this notebook.
  - Consider:
    - How much will the store's monthly revenue decrease (or increase) due to the change in price for the chosen Item (and its increased sales)?
    - How much will the store's monthly revenue increase (or decrease) due to the increased sales of the associated Items?
***

In [43]:
#Find name of product to discount
print(products_df[products_df['product_id'] == 13176])

       product_id            product_name  fake_unit_price
13175       13176  Bag of Organic Bananas              3.9


In [58]:
#Print out monthly revenue decrease due to change 10% decrease
count = 0
for i in orders_df['product_id']:
    if i == 13176:
        count+=1


discount_money = (count * 3.9 * -0.1) + (count * 0.05 * 3.9 * 0.9)
print('Total money lost from discount: $ ' + str(discount_money))
#Monthly revenue decrease is number of orders times price time ten percent decrease 
# plus the 5% increase in sales so number of orders times 5% times price times 100% - 10% price.

Total money lost from discount: $ -2496.5654999999997


In [54]:
#Find names and prices for all associated products.
discounts = []
index = 0
for i in rules_sorted['antecedents']:
    if next(iter(i)) == 13176:
        discounts.append(next(iter(rules_sorted['consequents'][index])))
    index+=1
for i in discounts:
    print(products_df[products_df['product_id'] == i])

       product_id          product_name  fake_unit_price
21136       21137  Organic Strawberries             1.01
       product_id          product_name  fake_unit_price
47208       47209  Organic Hass Avocado             5.46
       product_id          product_name  fake_unit_price
21902       21903  Organic Baby Spinach             7.17
       product_id         product_name  fake_unit_price
27965       27966  Organic Raspberries             6.59


In [56]:
#Compute 5% increase in sales for these associated items.
price = [1.01, 5.46, 7.17, 6.59]
total_discounted = 0
for i in range(len(discounts)):
    count = 0
    for i2 in orders_df['product_id']:
        if i2 == discounts[i]:
            count+=1
    total_discounted += count * price[i] * 0.05

print('Money generated from discount: $ ' + str(total_discounted)) 

Total money generated from discount: $ 5966.478999999999


In [60]:
print('Net dollars generated from discount: $' + str(total_discounted + discount_money))

Net dollars generated from discount: $3469.9134999999997


***
# Section: 6 - Conclusions
- Write a paragraph on what you discovered or learned from this homework.
***

I learned how to use mlxtend libraries like apriori and association rules. I learned how to manage memory and generate itemsets and rules. I ran out of memory many times generating itemsets and was stuck for a while. Professor Bihari helped with code on how to make one hot encoding and adding parameters to apriori. 

***
### END-OF-SUBMISSION
***